In [2]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd 
import numpy as np
import os
import torch
import torchvision
from torchvision import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
from torch.utils.data.dataloader import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from matplotlib import pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms, models
from PIL import Image
import numpy as np
import os
from torch.utils.data.sampler import SubsetRandomSampler
import pandas as pd
from torchinfo import summary



In [4]:
# Connexion à ggdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
working_dir = '/content/drive/MyDrive/M2_SISE/DL_MLProjet/Post-hurricane'

In [6]:
train_damage_dir = working_dir + '/train_another/damage'
validation_damage_dir = working_dir + '/validation_another/damage'
test_damage_dir = working_dir + '/test/damage'

train_nodamage_dir = working_dir + '/train_another/no_damage'
validation_nodamage_dir = working_dir + '/validation_another/no_damage'
test_nodamage_dir = working_dir + '/test/no_damage'

train_dir = working_dir + "/train_another"
valid_dir = working_dir + "/validation_another"

print('total training damage images: ',len(os.listdir(train_damage_dir)))
print('total validation damage images: ',len(os.listdir(validation_damage_dir)))
print('total test damage images: ',len(os.listdir(test_damage_dir)))

print('total training no damage images: ',len(os.listdir(train_nodamage_dir)))
print('total validation no damage images: ',len(os.listdir(validation_nodamage_dir)))
print('total test no damage images: ',len(os.listdir(test_nodamage_dir)))

total training damage images:  5000
total validation damage images:  1000
total test damage images:  1000
total training no damage images:  5000
total validation no damage images:  1000
total test no damage images:  1000


In [7]:
train = ImageFolder(train_dir,transform = transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))

valid = ImageFolder(valid_dir,transform = transforms.Compose([
    transforms.Resize((150,150)),transforms.ToTensor()
]))

print("Model's classes : \n",train.classes)

Model's classes : 
 ['damage', 'no_damage']


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.densenet121(pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=0.003)

In [14]:
train_loader = torch.utils.data.DataLoader(train, batch_size=32, shuffle=True, num_workers=2 , pin_memory = True)
valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=True, num_workers=2 , pin_memory = True)


In [15]:

        
n_classes = 2
model.fc = nn.Linear(in_features=2048, out_features=n_classes)


In [17]:
model.named_parameters()

<generator object Module.named_parameters at 0x7fee32f7ff90>

In [ ]:
model.train()
# Entrainement
for epoch in range(5):
    step_train = 0
    for inputs, labels in train_loader:
        # Propagation avant
        inputs.requires_grad_()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Propagation arrière 
        optimizer.zero_grad()

        for name,param in model.named_parameters():
          if "fc.weight" in name :
            param.requires_grad = True
          else :
            param.requires_grad = False

        loss.backward()
        optimizer.step()
        step_train += 1 
        #print(step_train)
        if step_train%20 == 0:
          print('Train / Step :', step_train)
        if step_train == 100:
          break
        
    # Validation
    accuracy = 0
    with torch.no_grad():
        step_val=0
        for inputs, labels in valid_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            accuracy += (predicted == labels).sum().item()
            step_val += 1 
            if step_val == 30:
              break
    accuracy = accuracy / len(valid)
    
    print("Epoch {}/20, Loss : {:.4f}, Accuracy : {:.2f}%".format(epoch+1, loss.item(), accuracy*100))




Model densenet 161

In [25]:
model2 = models.densenet161(pretrained=True)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model2.classifier.parameters(), lr=0.003)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/hub/checkpoints/densenet161-8d451a50.pth


  0%|          | 0.00/110M [00:00<?, ?B/s]

In [26]:
n_classes = 2
model2.fc = nn.Linear(in_features=2208, out_features=n_classes)

summary(model2)

Layer (type:depth-idx)                   Param #
DenseNet                                 --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       14,112
│    └─BatchNorm2d: 2-2                  192
│    └─ReLU: 2-3                         --
│    └─MaxPool2d: 2-4                    --
│    └─_DenseBlock: 2-5                  --
│    │    └─_DenseLayer: 3-1             101,952
│    │    └─_DenseLayer: 3-2             111,264
│    │    └─_DenseLayer: 3-3             120,576
│    │    └─_DenseLayer: 3-4             129,888
│    │    └─_DenseLayer: 3-5             139,200
│    │    └─_DenseLayer: 3-6             148,512
│    └─_Transition: 2-6                  --
│    │    └─BatchNorm2d: 3-7             768
│    │    └─ReLU: 3-8                    --
│    │    └─Conv2d: 3-9                  73,728
│    │    └─AvgPool2d: 3-10              --
│    └─_DenseBlock: 2-7                  --
│    │    └─_DenseLayer: 3-11            120,576
│    │    └─_DenseLayer: 3

In [ ]:
model2.train()
# Entrainement
for epoch in range(5):
    step_train = 0
    for inputs, labels in train_loader:
        # Propagation avant
        inputs.requires_grad_()
        outputs = model2(inputs)
        loss = criterion(outputs, labels)
        
        # Propagation arrière 
        optimizer.zero_grad()

        for name,param in model2.named_parameters():
          if "fc.weight" in name :
            param.requires_grad = True
          else :
            param.requires_grad = False

        loss.backward()
        optimizer.step()
        step_train += 1 
        #print(step_train)
        if step_train%20 == 0:
          print('Train / Step :', step_train)
        if step_train == 100:
          break
        
    # Validation
    accuracy = 0
    with torch.no_grad():
        step_val=0
        for inputs, labels in valid_loader:
            outputs = model2(inputs)
            _, predicted = torch.max(outputs.data, 1)
            accuracy += (predicted == labels).sum().item()
            step_val += 1 
            if step_val == 30:
              break
    accuracy = accuracy / len(valid)
    
    print("Epoch {}/20, Loss : {:.4f}, Accuracy : {:.2f}%".format(epoch+1, loss.item(), accuracy*100))